In [1]:
!pip install pyscf qiskit qiskit_ibm_runtime qiskit-addon-sqd ffsim -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.7/366.7 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.5 MB/s eta 0:00:00


In [2]:
import pyscf
import pyscf.cc
import pyscf.mcscf
import ffsim
import numpy as np
import matplotlib.pyplot as plt
import pickle
from typing import Sequence

import rustworkx
from qiskit.providers import BackendV2
from rustworkx import NoEdgeBetweenNodes, PyGraph

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [3]:
# Save your API key to track your progress and have access to the quantum computers

your_api_key = "ykLiHvC3SOFw5r1GB-ozI0YjYr9zD08JEnKlpT9YgI6G"
your_crn = "crn:v1:bluemix:public:quantum-computing:us-east:a/35f051d9a857434d90fa642e973c7520:884336a7-237f-4cfe-af9f-e05bbaf092cf::"


try:
  QiskitRuntimeService.save_account(
    channel="ibm_quantum_platform",
    token=your_api_key,
    instance=your_crn,
    name="luke",
  )
except:
  pass
service = QiskitRuntimeService(name="luke")
print(service.saved_accounts())

service.backends()

management.get:WARNING:2025-09-26 19:26:19,398: Loading saved account: luke


{'luke': {'channel': 'ibm_quantum_platform', 'url': 'https://cloud.ibm.com', 'token': 'ykLiHvC3SOFw5r1GB-ozI0YjYr9zD08JEnKlpT9YgI6G', 'instance': 'crn:v1:bluemix:public:quantum-computing:us-east:a/35f051d9a857434d90fa642e973c7520:884336a7-237f-4cfe-af9f-e05bbaf092cf::', 'verify': True, 'private_endpoint': False}}


[<IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_torino')>]

In [4]:
backend = service.least_busy(
    operational=True, simulator=False
)

print(f"Using backend {backend.name}")

Using backend ibm_brisbane


In [5]:
# --- add these two tiny helpers near the top (after imports) ---

import numpy as np

def _bitarray_to_np(meas):
    """Qiskit BitArray -> (shots, num_bits) uint8. Uses to_numpy() if available; else unpack bytes."""
    if hasattr(meas, "to_numpy"):
        arr = meas.to_numpy()
        return np.asarray(arr, dtype=np.uint8)
    # fallback: unpack packed bytes
    nbits = int(meas.num_bits)
    raw = np.asarray(getattr(meas, "_array"), dtype=np.uint8)  # BitArray stores packed bytes
    bits = np.unpackbits(raw, axis=-1, bitorder="little")[..., :nbits]
    return bits.astype(np.uint8)

def _meas_logical_order(transpiled_circ, bits_klass):
    """
    Reorder measured columns (classical-bit order) -> logical-qubit order (0..N-1).
    Uses the transpiled circuit itself: measure ops + .layout._p2v (physical->virtual).
    """
    # 1) classical-index -> physical-qubit index from measure ops
    cl2phys = {}
    for instr in transpiled_circ.data:
        if getattr(instr.operation, "name", None) != "measure":
            continue
        q = instr.qubits[0]
        c = instr.clbits[0]
        phys = transpiled_circ.find_bit(q).index
        cidx = transpiled_circ.find_bit(c).index
        cl2phys[cidx] = phys

    nbits = bits_klass.shape[1]
    if len(cl2phys) != nbits:
        raise ValueError(f"Expected {nbits} measured bits, got {len(cl2phys)}")

    # 2) physical -> virtual (logical) via layout._p2v; our original circuit used a single register 'q'
    layout = getattr(transpiled_circ, "layout", None)
    p2v = getattr(layout, "_p2v", None)
    if p2v is None:
        raise RuntimeError("Transpiled circuit has no layout._p2v; cannot build permutation.")

    # Build inverse index: inv[j] = classical column to read for logical qubit j
    inv = np.empty(nbits, dtype=int)
    for cidx in range(nbits):
        phys = cl2phys[cidx]
        vq = p2v[phys]                 # VirtualQubit
        logical_idx = vq.index         # since original circuit used QuantumRegister('q', 2*norb)
        inv[logical_idx] = cidx

    # Reorder columns into logical-qubit order
    return bits_klass[:, inv]


In [6]:
BASIS = "cc-pvdz"

# Approx geometries (Å): good enough for profiling
GEOMS = {
    "H2":  [("H",(0.0,0.0,0.0)), ("H",(0.74,0.0,0.0))],
    "H2O": [("O",(0.0,0.0,0.0)), ("H",(0.9572,0.0,0.0)), ("H",(-0.2390,0.9270,0.0))],  # ~104.5°
    "LiOH":[("Li",(0.0,0.0,0.0)), ("O",(1.73,0.0,0.0)), ("H",(2.69,0.0,0.0))],        # linear approx
    "CO":  [("C",(0.0,0.0,0.0)), ("O",(1.128,0.0,0.0))],
    "N2":  [("N",(0.0,0.0,0.0)), ("N",(1.10,0.0,0.0))],
}

# Freeze one core 1s *spatial* orbital per heavy atom (H:0, Li/C/N/O:1)
CORE_PER_ATOM = {"H":0, "Li":1, "C":1, "N":1, "O":1}

def n_frozen_from_geom(atom_list):
    return sum(CORE_PER_ATOM.get(sym, 0) for sym, _ in atom_list)

def build_case(name, basis=BASIS):
    atoms = GEOMS[name]
    spin = 0  # all closed-shell singlets here
    mol = pyscf.gto.Mole()
    mol.build(atom=atoms, basis=basis, spin=spin, unit="Angstrom", symmetry=False)

    # RHF (closed-shell)
    scf = pyscf.scf.RHF(mol).run()

    # Active space: freeze obvious 1s cores
    n_frozen = n_frozen_from_geom(atoms)
    active_space = range(n_frozen, mol.nao_nr())
    norb = len(active_space)

    # Electrons in the active block
    nelec_tot = int(sum(scf.mo_occ[active_space]))
    na = (nelec_tot + mol.spin)//2
    nb = (nelec_tot - mol.spin)//2

    # CASCI integrals in the active block
    cas = pyscf.mcscf.CASCI(scf, norb, (na, nb))
    mo = cas.sort_mo(active_space, base=0)
    h1, e_nuc = cas.get_h1cas(mo)
    # eri in physicist's notation, 4-index in active MO
    eri = pyscf.ao2mo.restore(1, cas.get_h2cas(mo), norb)

    return {
        "name": name,
        "h1": h1,                      # (norb, norb)
        "eri": eri,                    # (norb, norb, norb, norb) in 4-fold
        "e_nuc": e_nuc,                # scalar nuclear repulsion
        "norb": norb,
        "nelec": (na, nb),
        "n_frozen": n_frozen,
        "geom": atoms,
        "basis": basis,
        "scf": scf,
        "cas": cas,
        "active_space": active_space,
    }

def build_all(names=("H2","H2O","LiOH","CO","N2")):
    return {n: build_case(n) for n in names}

In [7]:
all_mols = build_all()

converged SCF energy = -1.12870009355644
converged SCF energy = -76.0267969728364
converged SCF energy = -82.9149376749489
converged SCF energy = -112.749311329757
converged SCF energy = -108.953796240891


In [8]:
for mol in all_mols.keys():
  # Get CCSD t2 amplitudes for initializing the ansatz
  ccsd = pyscf.cc.CCSD(
      all_mols[mol]['scf'], frozen=[i for i in range(all_mols[mol]['scf'].mol.nao_nr()) if i not in all_mols[mol]['active_space']]
  ).run()
  all_mols[mol]['ccsd'] = ccsd
  t1 = ccsd.t1
  all_mols[mol]['t1'] = t1
  t2 = ccsd.t2
  all_mols[mol]['t2'] = t2
  n_reps = 1
  num_orbitals = all_mols[mol]['norb']
  num_elec_a = all_mols[mol]['nelec'][0]
  num_elec_b = all_mols[mol]['nelec'][1]
  alpha_alpha_indices = [(p, p + 1) for p in range(num_orbitals - 1)]
  all_mols[mol]['alpha_alpha_indices'] = alpha_alpha_indices
  alpha_beta_indices = [(p, p) for p in range(0, num_orbitals, 4)]
  all_mols[mol]['alpha_beta_indices'] = alpha_beta_indices


  ucj_op = ffsim.UCJOpSpinBalanced.from_t_amplitudes(
      t2=t2,
      t1=t1,
      n_reps=n_reps,
      interaction_pairs=(alpha_alpha_indices, alpha_beta_indices),
      # Setting optimize=True enables the "compressed" factorization
      optimize=True,
      # Limit the number of optimization iterations to prevent the code cell from running
      # too long. Removing this line may improve results.
      options=dict(maxiter=1000),
  )
  all_mols[mol]['ucj_op'] = ucj_op

  nelec = (num_elec_a, num_elec_b)

  # create an empty quantum circuit
  qubits = QuantumRegister(2 * num_orbitals, name="q")
  all_mols[mol]['qubits'] = qubits
  circuit = QuantumCircuit(qubits)

  # prepare Hartree-Fock state as the reference state and append it to the quantum circuit
  circuit.append(ffsim.qiskit.PrepareHartreeFockJW(num_orbitals, nelec), qubits)

  # apply the UCJ operator to the reference state
  circuit.append(ffsim.qiskit.UCJOpSpinBalancedJW(ucj_op), qubits)
  circuit.measure_all()
  all_mols[mol]['circuit'] = circuit

E(CCSD) = -1.163374496404818  E_corr = -0.03467440284837669
E(CCSD) = -76.23799454061152  E_corr = -0.2111975677751045
E(CCSD) = -83.13001281450188  E_corr = -0.2150751395529544
E(CCSD) = -113.0437198131344  E_corr = -0.2944084833774329
E(CCSD) = -109.2635773150511  E_corr = -0.3097810741601993


In [9]:
IBM_TWO_Q_GATES = {"cx", "ecr", "cz"}

def create_linear_chains(num_orbitals: int) -> PyGraph:
    """In zig-zag layout, there are two linear chains (with connecting qubits between
    the chains). This function creates those two linear chains: a rustworkx PyGraph
    with two disconnected linear chains. Each chain contains `num_orbitals` number
    of nodes, that is, in the final graph there are `2 * num_orbitals` number of nodes.

    Args:
        num_orbitals (int): Number orbitals or nodes in each linear chain. They are
            also known as alpha-alpha interaction qubits.

    Returns:
        A rustworkx.PyGraph with two disconnected linear chains each with `num_orbitals`
            number of nodes.
    """
    G = rustworkx.PyGraph()

    for n in range(num_orbitals):
        G.add_node(n)

    for n in range(num_orbitals - 1):
        G.add_edge(n, n + 1, None)

    for n in range(num_orbitals, 2 * num_orbitals):
        G.add_node(n)

    for n in range(num_orbitals, 2 * num_orbitals - 1):
        G.add_edge(n, n + 1, None)

    return G


def create_lucj_zigzag_layout(
    num_orbitals: int, backend_coupling_graph: PyGraph
) -> tuple[PyGraph, int]:
    """This function creates the complete zigzag graph that 'can be mapped' to an IBM QPU with
    heavy-hex connectivity (the zigzag must be an isomorphic sub-graph to the QPU/backend
    coupling graph for it to be mapped).
    The zigzag pattern includes both linear chains (alpha-alpha interactions) and connecting
    qubits between the linear chains (alpha-beta interactions).

    Args:
        num_orbitals (int): Number of orbitals, that is, number of nodes in each alpha-alpha linear chain.
        backend_coupling_graph (PyGraph): The coupling graph of the backend on which the LUCJ ansatz
            will be mapped and run. This function takes the coupling graph as a undirected
            `rustworkx.PyGraph` where there is only one 'undirected' edge between two nodes,
            that is, qubits. Usually, the coupling graph of a IBM backend is directed (for example, Eagle devices
            such as ibm_brisbane) or may have two edges between two nodes (for example, Heron `ibm_torino`).
            A user needs to be make such graphs undirected and/or remove duplicate edges to make them
            compatible with this function.

    Returns:
        G_new (PyGraph): The graph with IBM backend compliant zigzag pattern.
        num_alpha_beta_qubits (int): Number of connecting qubits between the linear chains
            in the zigzag pattern. While we want as many connecting (alpha-beta) qubits between
            the linear (alpha-alpha) chains, we cannot accommodate all due to qubit and connectivity
            constraints of backends. This is the maximum number of connecting qubits the zigzag pattern
            can have while being backend compliant (that is, isomorphic to backend coupling graph).
    """
    isomorphic = False
    G = create_linear_chains(num_orbitals=num_orbitals)

    num_iters = num_orbitals
    while not isomorphic:
        G_new = G.copy()
        num_alpha_beta_qubits = 0
        for n in range(num_iters):
            if n % 4 == 0:
                new_node = 2 * num_orbitals + num_alpha_beta_qubits
                G_new.add_node(new_node)
                G_new.add_edge(n, new_node, None)
                G_new.add_edge(new_node, n + num_orbitals, None)
                num_alpha_beta_qubits = num_alpha_beta_qubits + 1
        isomorphic = rustworkx.is_subgraph_isomorphic(
            backend_coupling_graph, G_new
        )
        num_iters -= 1

    return G_new, num_alpha_beta_qubits


def lightweight_layout_error_scoring(
    backend: BackendV2,
    virtual_edges: Sequence[Sequence[int]],
    physical_layouts: Sequence[int],
    two_q_gate_name: str,
) -> list[list[list[int], float]]:
    """Lightweight and heuristic function to score isomorphic layouts. There can be many zigzag patterns,
    each with different set of physical qubits, that can be mapped to a backend. Some of them may
    include less noise qubits and couplings than others. This function computes a simple error score
    for each such layout. It sums up 2Q gate error for all couplings in the zigzag pattern (layout) and
    measurement of errors of physical qubits in the layout to compute the error score.

    Note:
        This lightweight scoring can be refined using concepts such as mapomatic.

    Args:
        backend (BackendV2): A backend.
        virtual_edges (Sequence[Sequence[int]]): Edges in the device compliant zigzag pattern where
            nodes are numbered from 0 to (2 * num_orbitals + num_alpha_beta_qubits).
        physical_layouts (Sequence[int]): All physical layouts of the zigzag pattern that are isomorphic
            to each other and to the larger backend coupling map.
        two_q_gate_name (str): The name of the two-qubit gate of the backend. The name is used for fetching
            two-qubit gate error from backend properties.

    Returns:
        scores (list): A list of lists where each sublist contains two items. First item is the layout, and
            second item is a float representing error score of the layout. The layouts in the `scores` are
            sorted in the ascending order of error score.
    """
    props = backend.properties()
    scores = []
    for layout in physical_layouts:
        total_2q_error = 0
        for edge in virtual_edges:
            physical_edge = (layout[edge[0]], layout[edge[1]])
            try:
                ge = props.gate_error(two_q_gate_name, physical_edge)
            except Exception:
                ge = props.gate_error(two_q_gate_name, physical_edge[::-1])
            total_2q_error += ge
        total_measurement_error = 0
        for qubit in layout:
            meas_error = props.readout_error(qubit)
            total_measurement_error += meas_error
        scores.append([layout, total_2q_error + total_measurement_error])

    return sorted(scores, key=lambda x: x[1])


def _make_backend_cmap_pygraph(backend: BackendV2) -> PyGraph:
    graph = backend.coupling_map.graph
    if not graph.is_symmetric():
        graph.make_symmetric()
    backend_coupling_graph = graph.to_undirected()

    edge_list = backend_coupling_graph.edge_list()
    removed_edge = []
    for edge in edge_list:
        if set(edge) in removed_edge:
            continue
        try:
            backend_coupling_graph.remove_edge(edge[0], edge[1])
            removed_edge.append(set(edge))
        except NoEdgeBetweenNodes:
            pass

    return backend_coupling_graph


def get_zigzag_physical_layout(
    num_orbitals: int, backend: BackendV2, score_layouts: bool = True
) -> tuple[list[int], int]:
    """The main function that generates the zigzag pattern with physical qubits that can be used
    as an `intial_layout` in a preset passmanager/transpiler.

    Args:
        num_orbitals (int): Number of orbitals.
        backend (BackendV2): A backend.
        score_layouts (bool): Optional. If `True`, it uses the `lightweight_layout_error_scoring`
            function to score the isomorphic layouts and returns the layout with less erroneous qubits.
            If `False`, returns the first isomorphic subgraph.

    Returns:
        A tuple of device compliant layout (list[int]) with zigzag pattern and an int representing
            number of alpha-beta-interactions.
    """
    backend_coupling_graph = _make_backend_cmap_pygraph(backend=backend)

    G, num_alpha_beta_qubits = create_lucj_zigzag_layout(
        num_orbitals=num_orbitals,
        backend_coupling_graph=backend_coupling_graph,
    )

    isomorphic_mappings = rustworkx.vf2_mapping(
        backend_coupling_graph, G, subgraph=True
    )
    isomorphic_mappings = list(isomorphic_mappings)

    edges = list(G.edge_list())

    layouts = []
    for mapping in isomorphic_mappings:
        initial_layout = [None] * (2 * num_orbitals + num_alpha_beta_qubits)
        for key, value in mapping.items():
            initial_layout[value] = key
        layouts.append(initial_layout)

    two_q_gate_name = IBM_TWO_Q_GATES.intersection(
        backend.configuration().basis_gates
    ).pop()

    if score_layouts:
        scores = lightweight_layout_error_scoring(
            backend=backend,
            virtual_edges=edges,
            physical_layouts=layouts,
            two_q_gate_name=two_q_gate_name,
        )

        return scores[0][0][:-num_alpha_beta_qubits], num_alpha_beta_qubits

    return layouts[0][:-num_alpha_beta_qubits], num_alpha_beta_qubits

In [10]:
for mol in all_mols.keys():
  print(mol)
  initial_layout, _ = get_zigzag_physical_layout(all_mols[mol]['norb'], backend=backend)

  pass_manager = generate_preset_pass_manager(
      optimization_level=3, backend=backend, initial_layout=initial_layout
  )

  # without PRE_INIT passes
  isa_circuit = pass_manager.run(all_mols[mol]['circuit'])
  print(f"Gate counts (w/o pre-init passes): {isa_circuit.count_ops()}")

  # with PRE_INIT passes
  # We will use the circuit generated by this pass manager for hardware execution
  pass_manager.pre_init = ffsim.qiskit.PRE_INIT
  isa_circuit = pass_manager.run(all_mols[mol]['circuit'])
  all_mols[mol]['isa_circuit'] = isa_circuit
  print(f"Gate counts (w/ pre-init passes): {isa_circuit.count_ops()}")

H2
Gate counts (w/o pre-init passes): OrderedDict({'rz': 1559, 'sx': 891, 'ecr': 436, 'x': 41, 'measure': 20, 'barrier': 1})
Gate counts (w/ pre-init passes): OrderedDict({'rz': 661, 'sx': 407, 'ecr': 180, 'measure': 20, 'x': 11, 'barrier': 1})
H2O
Gate counts (w/o pre-init passes): OrderedDict({'rz': 10343, 'sx': 9896, 'ecr': 2930, 'x': 66, 'measure': 46, 'barrier': 1})
Gate counts (w/ pre-init passes): OrderedDict({'sx': 5334, 'rz': 4978, 'ecr': 1452, 'measure': 46, 'x': 43, 'barrier': 1})
LiOH
Gate counts (w/o pre-init passes): OrderedDict({'rz': 18916, 'sx': 17995, 'ecr': 5378, 'x': 195, 'measure': 62, 'barrier': 1})
Gate counts (w/ pre-init passes): OrderedDict({'sx': 9480, 'rz': 8906, 'ecr': 2644, 'x': 122, 'measure': 62, 'barrier': 1})
CO
Gate counts (w/o pre-init passes): OrderedDict({'rz': 14267, 'sx': 13182, 'ecr': 4000, 'x': 138, 'measure': 52, 'barrier': 1})
Gate counts (w/ pre-init passes): OrderedDict({'sx': 6994, 'rz': 6528, 'ecr': 1896, 'x': 52, 'measure': 52, 'barrier'

In [11]:
all_mols['H2']['qubits']

QuantumRegister(20, 'q')

In [14]:
# for mol in all_mols.keys():
#   print(mol)
#   sampler = Sampler(mode=backend)
#   job = sampler.run([all_mols[mol]['isa_circuit']], shots=100_000)
#   from qiskit.providers.jobstatus import JobStatus
#   import time

#   while job.status() == 'QUEUED':
#     print('still queued')
#     time.sleep(30)
#   all_mols[mol]['primitive_result'] = job.result()
#   all_mols[mol]['pub_result'] = all_mols[mol]['primitive_result'][0]


for mol in all_mols.keys():
  print(mol)
  sampler = Sampler(mode=backend)
  job = sampler.run([all_mols[mol]['isa_circuit']], shots=100_000)

  from qiskit.providers.jobstatus import JobStatus
  import time
  while job.status() == 'QUEUED':
    print('still queued')
    time.sleep(30)

  primitive_result = job.result()
  pub_result = primitive_result[0]
  all_mols[mol]['primitive_result'] = primitive_result
  all_mols[mol]['pub_result'] = pub_result

  # --- Minimal, Qiskit 2.0–compatible bitstring canonicalization ---
  norb = int(all_mols[mol]['norb'])
  nbits = 2 * norb

  # 1) BitArray -> (shots, nbits) uint8
  raw_bits = pub_result.data.meas.to_bool_array(order="little").astype(np.uint8)

  if raw_bits.shape[-1] != nbits:
      raise ValueError(f"Expected {nbits} bits, got {raw_bits.shape[-1]}")

  tqc = all_mols[mol]['isa_circuit']

  # 2) classical index -> output-qubit position from measure ops
  cl2out = {}
  for instr in tqc.data:
      if getattr(instr.operation, "name", None) == "measure":
          outpos = tqc.find_bit(instr.qubits[0]).index   # position in output circuit
          cidx   = tqc.find_bit(instr.clbits[0]).index   # column in BitArray
          cl2out[cidx] = outpos
  if len(cl2out) != nbits:
      raise ValueError(f"Expected {nbits} measured bits, got {len(cl2out)}")

  # 3) output position -> input (logical) index via final_index_layout()
  #    final_index_layout()[j] = output position for input-qubit j  (Qiskit 2.0)
  outpos_for_in = tqc.layout.final_index_layout()        # length == nbits
  in_for_outpos  = {outpos: j for j, outpos in enumerate(outpos_for_in)}

  # 4) Build inverse permutation: new[:, j_logical] = raw[:, inv[j_logical]]
  inv = np.empty(nbits, dtype=int)
  for cidx, outpos in cl2out.items():
      j_logical = in_for_outpos[outpos]
      inv[j_logical] = cidx

  logical_bits = raw_bits[:, inv]                        # columns now in logical order (0..2*norb-1)

  # Save canonical logical-order bitstrings (already α-block then β-block given how you built the circuit)
  all_mols[mol]['bitstrings_block_ab'] = logical_bits




H2
still queued
H2O
still queued
LiOH
still queued
CO
still queued
N2
still queued


In [15]:
import pickle
pickle.dump(all_mols, open('all_mols.pkl', 'wb'))